In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
model_name = "meta-llama/Llama-3.2-1B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
device = "cpu"
model = model.to(device)

In [3]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm):

In [14]:
max_new_tokens = 10
prompt = "Is today before or after 2025?"

# output_dir = create_uuid_folder()
# os.makedirs(os.path.join(output_dir, "logits"), exist_ok=True)

# clear pre-existing hooks
for _, module in model.named_modules():
    module._forward_hooks.clear()
    module._backward_hooks.clear()

# format prompt
prompt = tokenizer.apply_chat_template(
    [{"role": "user", "content": prompt}], 
    tokenize=False, 
    add_generation_prompt=True, 
    return_tensors="pt"
)

# tokenize prompt
input_ids = tokenizer.encode(prompt, return_tensors='pt')
input_ids = input_ids.to(device)

In [15]:
embeddings = model.model.embed_tokens(input_ids)

In [16]:
embeddings.shape

torch.Size([1, 45, 2048])

In [23]:
embeddings[0][0].shape

torch.Size([2048])